# 如何创建会话

创建一个会话 （Session） 以使用 GraphScope。

会话封装了 GraphScope 的引擎的控制和状态。会话是 Python 客户端中的类，作为 GraphScope 的入口。用户可以使用会话在 K8s 集群上部署和连接到 GraphScope。

在这个教程中，我们会展示：

* 如何建立一个默认会话
* 建立会话的多种方式
* 如何将外部硬盘挂载到集群中

## 基础用法

首先，引入 graphscope 包。

In [ ]:
import graphscope

为了更好的理解部署过程，将 `show_log` 设置为 `True` 以获得详细的日志。

In [ ]:
graphscope.set_option(show_log=True)

即使没有任何参数，也可以简单的创建一个默认会话。

In [ ]:
s1 = graphscope.session()

命令执行后，会话会首先尝试去拉起协调者（Coordinator)，其为后端引擎的入口。协调者管理一个由 K8s Pod 组成的集群 （默认有二个 Pod），图交互引擎，图分析引擎，图学习引擎运行在集群上。对于集群中的每一个 Pod，都有一个 Vineyard 实例运行，提供内存中的分布式内存访问。

运行上面的代码单元格之后，输出的日志里包含创建会话的所有过程。

日志中 **GraphScope coordinator service connected** 代表会话创建成功，且当前 Python 客户端已连接到此会话。

如下命令也可以查看会话状态。

In [ ]:
s1

运行此单元格，可以看到 `“status": "active"` 的字样，代表会话状态正常。此外还有一些其他会话的元信息，如工作者 (Worker/Pod）数量，协调者的端点（endpoint）等等。

会话管理着集群的资源，因此在使用完会话后释放资源很重要。当图任务结束后使用 `session.close()` 来释放资源。

In [ ]:
s1.close()

## 进阶用法

GraphScope 中的会话提供了多个关键字参数来配置集群。

例如，`k8s_gs_image` 指定了 GraphScope 使用的镜像， `num_workers` 指定 Pod 的数量。

你可以使用 `help(graphscope.session）` 来查看会话的所有可配置项。 

In [ ]:
s2 = graphscope.session(num_workers=1, k8s_engine_cpu=1, k8s_engine_mem='4Gi', timeout_seconds=1200)

In [ ]:
s2.close()

也可以使用传入 Json 或者 字典 (`Dict`) 作为参数.

In [ ]:
config = {'num_workers': 1, 'timeout_seconds': 100}
s3 = graphscope.session(config=config)

In [ ]:
s3.close()

## 挂载路径

为了持久化数据，或从其他磁盘载入数据，会话提供了将文件路径挂载到集群中的功能。

例如，我们准备了一些图数据放到了宿主机上，位于 `/testingdata`，使用如下参数将其挂载到 Pod 中的 `/home/jovyan/datasets`上。

值得注意的是，服务器上的 `/testingdata` 路径只是当前 `/home/jovyan/datasets` 路径的一个 **拷贝** , 因此对应的修改不会影响到服务器上挂载的目录。


In [ ]:
k8s_volumes = {
    "data": {
        "type": "hostPath",
        "field": {
          "path": '/testingdata',
          "type": "Directory"
        },
        "mounts": {
          "mountPath": "/home/jovyan/datasets",
          "readOnly": True
        }
    }
}
s4 = graphscope.session(k8s_volumes=k8s_volumes)

In [ ]:
s4.close()